# Lbrerías

In [98]:
# General
import os
from typing import Tuple
import random
from collections import Counter
from PIL import Image
import time

# Pytorch
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision import datasets, transforms
import torchvision.models as models
import torch.optim as optim
from torch.quantization import QuantStub, DeQuantStub, get_default_qconfig, fuse_modules


# Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import WandbLogger

# Hydra
import hydra
from hydra.core.global_hydra import GlobalHydra
from omegaconf import DictConfig
from omegaconf import OmegaConf

# W&B
import wandb

# Sklearn
from sklearn.model_selection import train_test_split

# Matplotlib
import matplotlib.pyplot as plt

## Comprobacion de pytorch y CUDA

In [99]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("Device Name:", torch.cuda.get_device_name(0))

PyTorch version: 2.4.1
CUDA available: True
CUDA version: 11.8
Device Name: NVIDIA GeForce RTX 3050 Ti Laptop GPU


## Configuración de librerías

In [100]:
# Permite que PyTorch utilice Tensor Cores para operaciones de punto flotante en precisión mixta, lo que mejora significativamente el rendimiento en GPUs modernas.
# torch.set_float32_matmul_precision('high')

# Esto permite a CUDA optimizar dinámicamente las operaciones basándose en el tamaño de tus tensores.
# torch.backends.cudnn.benchmark = True

random.seed(42)

# Configuración de Weights & Biases
# user: alejandro-campos-technological-institute-of-costa-rica
wandb.login()

True

# Globales y constantes

In [101]:
#------------------------------------------------------------------------------------------------------------------------------------------------------------------
# Para la ejecución y corrida con los experimentos y configuraciones de hiperparámetros se utiliza Hydra, un framework open-source
# que simplifica el desarrollo de experimentos basado en una configuración jerárquica.
#------------------------------------------------------------------------------------------------------------------------------------------------------------------
GlobalHydra.instance().clear()
hydra.initialize(config_path=".", version_base=None)
cfg = hydra.compose(config_name="config")

print(OmegaConf.to_yaml(cfg))

CHECKPOINTS_PATH = 'checkpoints'

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM_WORKERS = 4

model:
  learning_rate: 0.003
  batch_size: 16
  epochs: 2
  patience: 3
  train_dataset_dir: data/train
  validation_dataset_dir: data/valid
  test_dataset_dir: data/test
data:
  labeled_percentage: 0.3



# Definiciones generales

## Clase de carga

Debe crear su propia clase de carga de datos utilizando “LightningDataModule”

In [102]:
class CustomLoader(pl.LightningDataModule):
    def __init__(self, dataset: Dataset, batch_size=32):
        super().__init__()
        self.dataset = dataset
        self.batch_size = batch_size


    def split(self, labeled_percentage) -> Tuple[Dataset, Dataset]:
        labels = [sample[1] for sample in self.dataset.samples]        
        
        # División estratificada
        labeled_indices, unlabeled_indices = train_test_split \
        (
            range(len(self.dataset)),
            test_size=1 - labeled_percentage,
            stratify=labels
        )

        labeled_subset = Subset(self.dataset, labeled_indices)
        unlabeled_subset = Subset(self.dataset, unlabeled_indices)
        
        labeled_dataset = self.create_dataset_from_subset(labeled_subset)
        unlabeled_dataset = self.create_dataset_from_subset(unlabeled_subset)

        return labeled_dataset, unlabeled_dataset


    def create_dataset_from_subset(self, subset):
        samples = [self.dataset.samples[idx] for idx in subset.indices]
        new_dataset = datasets.ImageFolder(root=self.dataset.root, transform=self.dataset.transform)
        new_dataset.samples = samples
        new_dataset.targets = [sample[1] for sample in samples]  # Actualizar etiquetas
        return new_dataset


    def get_dataloader(self):
        return DataLoader(self.labeled_data, batch_size=self.batch_size, shuffle=True)

## Modelo del Autoencoder

Creación de un propio modelo utilizando “LightningModule” basado en la Arquitectura U-Net con Skip Connections.

In [103]:
class UNetAutoencoder(pl.LightningModule):
    def __init__(self, input_channels=3, output_channels=3, learning_rate=1e-3):
        super(UNetAutoencoder, self).__init__()
        self.learning_rate = learning_rate

        # Encoder
        self.enc1 = self.conv_block(input_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        
        # Bottleneck
        self.bottleneck = self.conv_block(512, 1024)
        
        # Decoder
        self.dec4 = self.upconv_block(1024, 512)
        self.dec3 = self.upconv_block(512 + 512, 256)
        self.dec2 = self.upconv_block(256 + 256, 128)
        self.dec1 = self.upconv_block(128 + 128, 64)

        # Final layer
        self.final_layer = nn.Conv2d(128, output_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        return nn.Sequential \
        (
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )


    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential \
        (
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True)
        )
    
    
    def forward(self, x):
        # Encoder
        # print("\nx:", x.shape)  # Muestra las dimensiones de la entrada
        enc1 = self.enc1(x)
        # print("enc1 size:", enc1.shape)  # Muestra las dimensiones después de la primera capa

        enc2 = self.enc2(F.max_pool2d(enc1, 2))
        # print("enc2 size:", enc2.shape)  # Muestra las dimensiones después de la segunda capa

        enc3 = self.enc3(F.max_pool2d(enc2, 2))
        # print("enc3 size:", enc3.shape)  # Muestra las dimensiones después de la tercera capa

        enc4 = self.enc4(F.max_pool2d(enc3, 2))
        # print("enc4 size:", enc4.shape)  # Muestra las dimensiones después de la cuarta capa

        bottleneck = self.bottleneck(F.max_pool2d(enc4, 2))
        # print("bottleneck size:", bottleneck.shape)  # Muestra las dimensiones del cuello de botella


        # Decoder
        dec4 = self.dec4(bottleneck)
        # print("dec4 size:", dec4.shape)  # Muestra las dimensiones después de la primera capa del decodificador
        dec4 = torch.cat((dec4, enc4), dim=1)
        # print("dec4 after concatenation size:", dec4.shape)  # Después de concatenar

        dec3 = self.dec3(dec4)
        # print("dec3 size:", dec3.shape)  # Muestra las dimensiones después de la segunda capa del decodificador
        dec3 = torch.cat((dec3, enc3), dim=1)
        # print("dec3 after concatenation size:", dec3.shape)  # Después de concatenar

        dec2 = self.dec2(dec3)
        # print("dec2 size:", dec2.shape)  # Muestra las dimensiones después de la tercera capa del decodificador
        dec2 = torch.cat((dec2, enc2), dim=1)
        # print("dec2 after concatenation size:", dec2.shape)  # Después de concatenar

        dec1 = self.dec1(dec2)
        # print("dec1 size:", dec1.shape)  # Muestra las dimensiones después de la cuarta capa del decodificador
        dec1 = torch.cat((dec1, enc1), dim=1)
        # print("dec1 after concatenation size:", dec1.shape)  # Después de concatenar
        
        # Output layer
        output = self.final_layer(dec1)
        # print("Output size:", output.shape)  # Muestra las dimensiones de la salida
        return output
    


    #------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # Debe redefinir como mínimo los métodos de training_step, test_step, configure_optimizers
    #------------------------------------------------------------------------------------------------------------------------------------------------------------------
    def training_step(self, batch, batch_idx):
        inputs, _ = batch  # Ignora las etiquetas
        outputs = self(inputs)
        loss = F.mse_loss(outputs, inputs)
        self.log('train_loss', loss)
        return loss

    
    def test_step(self, batch, batch_idx):
        inputs, _ = batch  # Ignora las etiquetas
        outputs = self(inputs)
        loss = F.mse_loss(outputs, inputs)
        self.log('test_loss', loss)
        return loss
    

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

### Función de entrenamiento

Se utiliza el Callback de EarlyStopping durante el proceso de entrenamiento para evitar Overfitting del modelo.

In [104]:
def wandb_train(model: UNetAutoencoder, train_data_loader: DataLoader, model_name: str, wandb_project_name: str):
    wandb_logger = WandbLogger \
    (
        project=wandb_project_name,
        name=model_name,
        log_model=True  # Guarda el modelo en Wandb.
    )
    
    trainer = pl.Trainer(
        max_epochs=cfg.model.epochs,
        accelerator="gpu",  # Especifica que se usará GPU
        devices="auto",  # Usa todas las GPUs disponibles
        precision=16,  # Habilita entrenamiento en precisión mixta (16 bits)
        logger=wandb_logger,
        callbacks=[EarlyStopping(monitor="train_loss", patience=int(cfg.model.patience), mode="min")]
    )

    try:
        trainer.fit(model, train_data_loader)
        filepath = os.path.join(CHECKPOINTS_PATH, f'{model_name}.ckpt')
        trainer.save_checkpoint(filepath)
    finally:
        wandb.finish()

### Función de checkpoint

In [105]:
def checkpoint_UNetAutoencoder(model, model_name: str, ):
    filepath = os.path.join(CHECKPOINTS_PATH, f'{model_name}.ckpt')

    if os.path.exists(filepath):
        model = UNetAutoencoder.load_from_checkpoint(filepath)
        print("-------------------------------------------------------------")
        print(f"Checkpoint {model_name} cargado con éxito.")
    else:
        print(f"Error: No se encontró el archivo de checkpoint en {filepath}.")

## Clasificadores

### Clasificador A

El clasificador A será entrenado desde 0

In [106]:
# Clasificador A
class ClassifierA(nn.Module):
    def __init__(self, num_classes=20):
        super(ClassifierA, self).__init__()

        # Usamos el modelo ResNet50 preentrenado como base
        self.model = models.resnet50(pretrained=False)

        # Ajustar la última capa
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

### Clasificador B

In [107]:
class ClassifierB(nn.Module):
    def __init__(self, encoder, num_classes=20):
        super(ClassifierB, self).__init__()

        self.encoder = encoder
        self.fc = nn.Sequential \
        (
            nn.Flatten(),
            nn.Linear(512 * 16 * 16, 256),  # Ajusta dimensiones según la salida del encoder
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        # print(f'entrada del modelo B: {x.shape}')
        x = self.encoder(x)
        # print(f'salida del encoder: {x.shape}')
        x = self.fc(x)
        # print(f'salida de la fc: {x.shape}')
        return x

#### Encoder (Para los modelos B)

In [108]:
class Encoder(nn.Module):
    def __init__(self, autoencoder: UNetAutoencoder):
        super(Encoder, self).__init__()
        self.enc1 = autoencoder.enc1
        self.enc2 = autoencoder.enc2
        self.enc3 = autoencoder.enc3
        self.enc4 = autoencoder.enc4

    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(F.max_pool2d(x, 2))
        x = self.enc3(F.max_pool2d(x, 2))
        x = self.enc4(F.max_pool2d(x, 2))
        return x

### Funciones auxiliares

In [109]:
def checkpoint_classifier(model, model_name: str, save=False):
    filepath = os.path.join(CHECKPOINTS_PATH, f'{model_name}.ckpt')
    
    if save:
        torch.save(model.state_dict(), filepath)
        print(f"Checkpoint guardado en {filepath}.")
    else:
        if os.path.exists(filepath):
            model.load_state_dict(torch.load(filepath))
            print("-------------------------------------------------------------")
            print(f"Checkpoint {model_name}.ckpt cargado con éxito.")
        else:
            print(f"Error: No se encontró el archivo de checkpoint en {filepath}.")


def train_clasifier(clasifier_model, model_name: str, train_dataloader: DataLoader, num_epochs=cfg.model.epochs):
    print(f'Training: {model_name}')
    clasifier_model = clasifier_model.to(DEVICE)

    optimizer = optim.Adam(clasifier_model.parameters(), lr=cfg.model.learning_rate)
    criterion = nn.CrossEntropyLoss()


    for epoch in range(num_epochs):

        clasifier_model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()            
            outputs = clasifier_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Estadísticas
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # torch.cuda.empty_cache()

        # Imprimir estadísticas del epoch
        train_loss = running_loss / len(train_dataloader)
        train_accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")

    checkpoint_classifier(model=clasifier_model, model_name=model_name, save=True)


def cuantization(clasifier_model):
    model_quantized = clasifier_model.__class__()  # Crea una instancia vacía del mismo tipo de modelo
    model_quantized.load_state_dict(clasifier_model.state_dict())  # Cargar los pesos del modelo original
    model_quantized = model_quantized.to('cpu')
    model_quantized.eval()

    model_quantized.qconfig = torch.quantization.get_default_qconfig('fbgemm')
    torch.quantization.prepare(model_quantized, inplace=True)
    torch.quantization.convert(model_quantized, inplace=True)
    
    print(model_quantized)
    return model_quantized

# Manipulación de datos

## Carga de las muestras

Se seleccionan solamente las 20 especies con mayor cantidad de muestras. Además se mueven unas 20 muestras del set de entrenamiento de cada especie al set de testing para incrementar su volumen, porque actualmente solo tiene 5 muestras para testing.

In [110]:
# El dataset no redimensiona las imágenes de forma inmediata al ser cargadas en memoria. Las imágenes se cargan solo cuando accedes a ellas.
# Las transformaciones se aplican cada vez que accedes a una imagen, no al momento de la creación del dataset. Esto significa que la imagen será
# redimensionada solo cuando el dataset realmente cargue la imagen durante el entrenamiento o la evaluación (cuando haces una iteración sobre el dataset).

train_full_dataset = datasets.ImageFolder(root=cfg.model.train_dataset_dir)
test_full_dataset = datasets.ImageFolder(root=cfg.model.test_dataset_dir)

train_species_counts = Counter([sample[1] for sample in train_full_dataset.samples])
top_20_species = [species for species, count in train_species_counts.most_common(20)]

train_samples_from_top_20_species_by_image_count = [sample for sample in train_full_dataset.samples if sample[1] in top_20_species]
test_samples_from_top_20_species_by_image_count = [sample for sample in test_full_dataset.samples if sample[1] in top_20_species]

train_samples = []
test_samples = test_samples_from_top_20_species_by_image_count

for specie in top_20_species:
    species_samples = [sample for sample in train_samples_from_top_20_species_by_image_count if sample[1] == specie] # if sample tag = tag
    random.shuffle(species_samples)
    train_samples.extend(species_samples[:-20])
    test_samples.extend(species_samples[-20:])


def remap_indices(samples):
    unique_indices = sorted({label for _, label in samples})
    index_map = {original: remapped for remapped, original in enumerate(unique_indices)}    
    remapped_samples = [(path, index_map[label]) for path, label in samples]
    return remapped_samples


train_samples = remap_indices(train_samples)
test_samples = remap_indices(test_samples)

print(f'Training samples: {len(train_samples)}')
print(f'Testing samples: {len(test_samples)}')


print(train_samples)
print(test_samples)

Training samples: 2546
Testing samples: 500
[('data/train\\MOURNING CLOAK\\003.jpg', 13), ('data/train\\MOURNING CLOAK\\067.jpg', 13), ('data/train\\MOURNING CLOAK\\013.jpg', 13), ('data/train\\MOURNING CLOAK\\062.jpg', 13), ('data/train\\MOURNING CLOAK\\100.jpg', 13), ('data/train\\MOURNING CLOAK\\037.jpg', 13), ('data/train\\MOURNING CLOAK\\177.jpg', 13), ('data/train\\MOURNING CLOAK\\070.jpg', 13), ('data/train\\MOURNING CLOAK\\054.jpg', 13), ('data/train\\MOURNING CLOAK\\014.jpg', 13), ('data/train\\MOURNING CLOAK\\158.jpg', 13), ('data/train\\MOURNING CLOAK\\016.jpg', 13), ('data/train\\MOURNING CLOAK\\153.jpg', 13), ('data/train\\MOURNING CLOAK\\053.jpg', 13), ('data/train\\MOURNING CLOAK\\128.jpg', 13), ('data/train\\MOURNING CLOAK\\045.jpg', 13), ('data/train\\MOURNING CLOAK\\149.jpg', 13), ('data/train\\MOURNING CLOAK\\043.jpg', 13), ('data/train\\MOURNING CLOAK\\006.jpg', 13), ('data/train\\MOURNING CLOAK\\077.jpg', 13), ('data/train\\MOURNING CLOAK\\176.jpg', 13), ('data/tra

## Creación de los transforms

In [111]:
transform = transforms.Compose \
(
    [
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

## Creación de Datasets

In [112]:
# Set de datos de entrenamiento
train_dataset = datasets.ImageFolder(root=cfg.model.train_dataset_dir, transform=transform)
# Sobreescritura de los samples
train_dataset.samples = train_samples

# Set de datos de prueba
test_dataset = datasets.ImageFolder(root=cfg.model.test_dataset_dir, transform=transform)
# Sobreescritura de los samples
test_dataset.samples = test_samples

## Creación de los Dataloaders

Se toma el set de datos de entrenamiento y se simula que una parte de cada clase no contiene labels, denominado `set_de_datos_sin_labels`, el otro restante será denominado `set_de_datos_con_labels`.

In [113]:
custom_loader = CustomLoader(dataset=train_dataset, batch_size=cfg.model.batch_size)
set_de_datos_con_labels, set_de_datos_sin_labels = custom_loader.split(labeled_percentage=cfg.data.labeled_percentage)

print(f"Labeled: {len(set_de_datos_con_labels.samples)}")
print(f"Unlabeled: {len(set_de_datos_sin_labels.samples)}")

# Dataloaders
train_labeled_dl = DataLoader(dataset=set_de_datos_con_labels, batch_size=cfg.model.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
train_unlabeled_dl = DataLoader(dataset=set_de_datos_sin_labels, batch_size=cfg.model.batch_size, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

Labeled: 763
Unlabeled: 1783


`pin_memory=True` Optimiza la transferencia de datos entre la CPU y la GPU

# Experimento 1

## Instanciación y entrenamiento del Autoencoder

In [114]:
model_exp_1 = UNetAutoencoder(input_channels=3, output_channels=3, learning_rate=cfg.model.learning_rate)
model_exp_1 = model_exp_1.to(DEVICE)
# wandb_train(model=model_exp_1, train_data_loader=train_unlabeled_dl, model_name="model_exp_1", wandb_project_name="Experimento_1")

## Checkpoint Autoencoder

In [115]:
# Instanciación del modelo sin entrenar
model_exp_1 = UNetAutoencoder(input_channels=3, output_channels=3, learning_rate=cfg.model.learning_rate)
print("Pesos iniciales:")
print(model_exp_1.enc1[0].weight)


checkpoint_UNetAutoencoder(model=model_exp_1, model_name='model_exp_1')

Pesos iniciales:
Parameter containing:
tensor([[[[ 0.1645, -0.1714,  0.1442],
          [ 0.1814, -0.0365,  0.0752],
          [ 0.1304, -0.0769, -0.0346]],

         [[-0.0389, -0.1190,  0.0969],
          [ 0.0476,  0.0381,  0.0190],
          [-0.0742, -0.1620, -0.1294]],

         [[-0.0268,  0.0910,  0.0066],
          [ 0.1347, -0.0266, -0.0250],
          [ 0.0964, -0.1129, -0.0092]]],


        [[[-0.1768, -0.1556, -0.1492],
          [ 0.1082, -0.1828,  0.1224],
          [-0.0663, -0.0564,  0.0031]],

         [[ 0.0626,  0.0216, -0.0383],
          [ 0.0903,  0.1685, -0.1670],
          [ 0.0309,  0.1298,  0.0063]],

         [[-0.0146,  0.0762,  0.1150],
          [ 0.1513,  0.1468, -0.0619],
          [-0.1887, -0.0352, -0.0448]]],


        [[[-0.0992,  0.1313, -0.1114],
          [ 0.1904,  0.0969, -0.0876],
          [ 0.1778, -0.0346,  0.0744]],

         [[-0.1556, -0.1044, -0.0838],
          [-0.1148,  0.1285, -0.1557],
          [ 0.0459, -0.1124,  0.1666]],

     

## Reconstrucción de imágenes usando el Autoencoder

Se utiliza el `set_de_datos_sin_labels` para reconstruir las muestras.

In [116]:
def test_reconstruction():
    # Al parecer algo mueve el modelo al CPU en algun punto luego del entrenamiento
    print(f'El modelo está en: {next(model_exp_1.parameters()).device}')

    # Verificar si el modelo está en CPU y moverlo a GPU si es necesario
    if next(model_exp_1.parameters()).device.type == 'cpu' and DEVICE.type == 'cuda':
        print('Moviendo el modelo al GPU...')
        model_exp_1 = model_exp_1.to(DEVICE)

    # Verificar que las imágenes estén en el dispositivo correcto (GPU)
    data_iter = iter(train_unlabeled_dl)
    images, labels = next(data_iter)
    images = images.to(DEVICE)

    # Realizar la inferencia sin calcular los gradientes lo cual ahorra uso de memoria
    with torch.no_grad():
        reconstructed_images = model_exp_1(images)

    original_image = images[0].cpu().detach().numpy().transpose(1, 2, 0)
    reconstructed_image = reconstructed_images[0].cpu().detach().numpy().transpose(1, 2, 0)

    # Mostrar una imagen original junto a una reconstruida
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.imshow(original_image)
    plt.title("Imagen Original")
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(reconstructed_image)
    plt.title("Imagen Reconstruida")
    plt.axis('off')

    plt.show()

# test_reconstruction()

## Instanciación y entrenamiento del calsificador A

In [117]:
model_A = ClassifierA()

# train_clasifier(clasifier_model=model_A, model_name='model_A', train_dataloader=train_labeled_dl)
checkpoint_classifier(model=model_A, model_name='model_A', save=False)

c:\Users\aleja\.conda\envs\env_IC6200_AI_P2_pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\aleja\.conda\envs\env_IC6200_AI_P2_pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


-------------------------------------------------------------
Checkpoint model_A.ckpt cargado con éxito.


C:\Users\aleja\AppData\Local\Temp\ipykernel_6172\3195659470.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filepath))


## Instanciación de los modelos B

In [118]:
encoder = Encoder(model_exp_1)

for param in encoder.parameters():
    param.requires_grad = False

model_B1 = ClassifierB(encoder)
model_B2 = ClassifierB(Encoder(model_exp_1))

## Entrenamiento de los modelos B

In [119]:
# train_clasifier(clasifier_model=model_B1, model_name='model_B1', train_dataloader=train_labeled_dl)
# train_clasifier(clasifier_model=model_B2, model_name='model_B2', train_dataloader=train_labeled_dl)

checkpoint_classifier(model=model_B1, model_name='model_B1', save=False)
checkpoint_classifier(model=model_B2, model_name='model_B2', save=False)

C:\Users\aleja\AppData\Local\Temp\ipykernel_6172\3195659470.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(filepath))


-------------------------------------------------------------
Checkpoint model_B1.ckpt cargado con éxito.
-------------------------------------------------------------
Checkpoint model_B2.ckpt cargado con éxito.


## Cuantización de los modelos A y B (B1 y B2)

In [120]:
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso
# En proceso

# print('--- Modelo A Normal ---')
# print(model_A)
# print()

# print('--- Cuantizacion del Modelo A ---')
# quantized_model_A = cuantization(clasifier_model=model_A, train_dataloader=train_labeled_dl)

# print('--- Modelo A Normal ---')
# print(model_A)
# print()

# # print('--- Cuantizacion del Modelo B1 ---')
# # quantized_model_B1 = cuantization(clasifier_model=model_B1, train_dataloader=train_labeled_dl)
# # print()

# # print('--- Cuantizacion del Modelo B2 ---')
# # quantized_model_B2 = cuantization(clasifier_model=model_B2, train_dataloader=train_labeled_dl)
# # print()